# 在后台执行微调py文件的时候，程序老是中断，暂时不提交结果，后面会补上的。

In [ ]:
import os
import subprocess

# 设置 HF_ENDPOINT 环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

def download_model(model_name):
    try:
        subprocess.run(['huggingface-cli', 'download', '--resume-download', model_name,'--local-dir',model_name], check=True)
        print(f"Model '{model_name}' downloaded successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading model '{model_name}': {e}")

def download_dataset(dataset_name):
    try:
        subprocess.run(['huggingface-cli', 'download', '--resume-download','--repo-type','dataset',dataset_name,"--local-dir",dataset_name], check=True)
        print(f"Dataset '{dataset_name}' downloaded successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading dataset '{dataset_name}': {e}")

In [3]:
download_dataset("yelp_review_full")

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]downloading https://hf-mirror.com/datasets/yelp_review_full/resolve/c1f9ee939b7d05667af864ee1cb066393154bf85/.gitattributes to /root/.cache/huggingface/hub/datasets--yelp_review_full/blobs/957b2579c6ef20995a09efd9a17f8fd90606f5ed.incomplete
downloading https://hf-mirror.com/datasets/yelp_review_full/resolve/c1f9ee939b7d05667af864ee1cb066393154bf85/yelp_review_full/test-00000-of-00001.parquet to /root/.cache/huggingface/hub/datasets--yelp_review_full/blobs/bf06d5969bff93ecd4a6d4b330643761ce108b42e9b8a894cf82c9df02a08540.incomplete
downloading https://hf-mirror.com/datasets/yelp_review_full/resolve/c1f9ee939b7d05667af864ee1cb066393154bf85/README.md to /root/.cache/huggingface/hub/datasets--yelp_review_full/blobs/dbd7ee1cd254de795be0620f97800f28615df66e.incomplete



/mnt/workspace/new2/LLM-quickstart/transformers/yelp_review_full
Dataset 'yelp_review_full' downloaded successfully.



train-00000-of-00001.parquet: 100%|██████████| 299M/299M [00:35<00:00, 8.37MB/s]
Fetching 4 files: 100%|██████████| 4/4 [00:37<00:00,  9.49s/it]


In [6]:
from datasets import load_dataset

dataset = load_dataset("/mnt/workspace/yelp_review_full/yelp_review_full/0.0.0")

Generating train split: 650000 examples [00:00, 1273688.13 examples/s]
Generating test split: 50000 examples [00:00, 1259974.53 examples/s]


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [13]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [14]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [15]:
show_random_elements(dataset["train"])

,label,text
0,4 stars,"We had a late, light dinner here tonight. It was recommended by our B&B host and is an easy block and a half walk. The building is old and the d\u00e9cor is charming and eclectic. The room you enter has the bar and it's noisier than the room to the left. We were greeted warmly and given our choice of tables. We chose the quieter room. Tables are covered w/ butcher paper. There's a bilingual menu and the table d'h\u00f4te is on a blackboard. Many things looked good, but we weren't hungry enough for anything major. \n\nAlan chose the smoked salmon entr\u00e9e (entr\u00e9e here is an appetizer at home). Our hostess was concerned that it wouldn't be enough for him. I chose a four cheese sandwich w/ tomato and pesto. It was heated but not grilled as a panini is. It came w/ a choice of fries or salad and I chose the latter. Both salad and sandwich were tasty. Alan was pleased w/ his salmon. He had bites of the salad and of the sandwich, so we both had enough food because I wasn't as hungry as he was. We both had water to drink. Service was friendly and accommodating and the food hit the spot. 3.51, rounded up."
1,1 star,"Not a fan. This place has a musty smell when you walk in, until you get to the bar. Then, it smells like a dirty bar mat that needs to be cleaned. The barstools are also very uncomfortable. Leaving forward and made me feel like I was going to fall out of it. \n\nIt was pretty unpleasant so we left without ordering any food therefore I cannot speak to the food quality."
2,3 stars,"I'd call this solid. Good execution of the high-end fundamentals (Federal Hill Style Calamari, Sunday Gravy, Squid Ink Risotto with Scallops...), but nothing truly Babbo-new to make you rethink the classics.\n\nAgain... Solid, authentic Italian for Nevada, pricey for Boston, but on balance I'd come back."
3,3 stars,"Getting here for Spring Training was hell in hand basket. It probably took us 25 minutes from the time we turned onto Bullard to finally get a parking space. The layout of the place is not conducive at all to efficient flow of traffic. Once we parked it was a hike to get to the stadium itself so note to self for Spring Training next year:\""Leave two hours before the game starts\""\nOnce we got to the stadium it was nice enough. We had seats by third base which weren't bad but this place is a little bigger than the other places I hit so I wasn't impressed by the seats. They have good food options at the concession stand and they have batting cages and a merry-go-round for kids which was good for me because an obnoxious brat was sitting behind me kicking my seat and being loud. I can only assume that his parents, after finally realizing that their kid was pissing me off, took him to the merry go round....otherwise I would have used him for batting practice. \nThe entire complex itself is nice:they have a park and a pool and of course the stadium itself. I imagine it a nice place to come for a family outing....just not during Spring Training."
4,5 stars,I wasn't even planning on buying a vape... My boyfriend went in to get one but everyone there was so nice and friendly (even their cute pup!) and there were soooo many good flavors I ended up buying one! The small starter one... They help me put it together & everything. Great experience :)
5,4 stars,"I went here with my wife and couple of our friends during the Sunday morning breakfast rush. This place was very busy, and I could see why. The reason? Bacon. Need I say more?\n\nSo I totally pigged out. I figures, what the heck. New place? Let's go knee deep. \n\nI had The Oink Omelette. I used to be a fat kid, so I felt my old self pop out. \n\nFive eggs, with bacon, sausage, ham and cheddar cheese. With hash browns. How I didn't keel over with a heart attack, I don't know. In my defense! I didn't eat the whole thing! \n\nBut I did eat a lot. The servers here were so friendly! There was an older woman in particular who I noticed calling people by their names. 

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 165kB/s]
config.json: 570B [00:00, 2.77MB/s]                   
vocab.txt: 213kB [00:00, 1.67MB/s]
tokenizer.json: 436kB [00:00, 1.14MB/s]
Map: 100%|██████████| 50000/50000 [00:17<00:00, 2828.60 examples/s]


In [17]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"Stop #1 for the Light Rail Pub Crawl was Rula Bula... at least for me - and it was my first time there!\n\nI was with my mother-in-law-to-be (Joan M), and upon making our way to the back patio, we were welcomed by a huge group of yelpers. Great back patio by the way. I had a Stella and Joan an Irish coffee...\n\nKate was our server and handled our group well - she did a good job considering so many peeps arriving at different times and paying separately. Only one discrepancy that I overheard but it wasn't too big a deal...\n\nThe bathroom was pretty filthy and the TP was out in one stall while the door wouldn't shut on another - not a fan of nasty bathrooms in restaurants.\n\nEnjoyable pub atmosphere and convenient location on Mill Ave just off the Light Rail. Although we didn't eat during our visit, I was impressed with the selection on the menu and based on the positive reviews that came before mine, I would totally be open to returning to eat sometime. I'll update then!","[101, 6682, 108, 122, 1111, 1103, 3935, 8654, 21385, 140, 28115, 1233, 1108, 155, 5886, 139, 5886, 119, 119, 119, 1120, 1655, 1111, 1143, 118, 1105, 1122, 1108, 1139, 1148, 1159, 1175, 106, 165, 183, 165, 183, 2240, 1108, 1114, 1139, 1534, 118, 1107, 118, 1644, 118, 1106, 118, 1129, 113, 6914, 150, 114, 117, 1105, 1852, 1543, 1412, 1236, 1106, 1103, 1171, 22408, 117, 1195, 1127, 10977, 1118, 170, 3321, 1372, 1104, 6798, 1233, 6206, 119, 2038, 1171, 22408, 1118, 1103, 1236, 119, 146, 1125, 170, 12126, 1105, 6914, 1126, 2600, 3538, 119, 119, 119, 165, 183, 165, 183, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [20]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [23]:
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

In [26]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

2024-01-28 19:00:54.157032: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:00:54.204448: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 19:00:54.204474: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 19:00:54.204499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-28 19:00:54.213174: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 19:00:54.213583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate(small_test_dataset)

In [ ]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [ ]:
trainer.save_state()